**TASK 1**

In [179]:
import requests
import lxml.html as lh
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('///tr')

#Check the length of the rows
print('Quantity of columns:',[len(T) for T in tr_elements[:1]])

Quantity of columns: [3]


In [180]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postal Code

2 Borough

3 Neighborhood



In [181]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [182]:
print('Number of rows',[len(C) for (title,C) in col])

Number of rows [181, 181, 181]


In [183]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [184]:
##Remove N/A Borough rows
df.rename(columns={'Postal Code\n':'Postal Code'}, inplace=True)
df.rename(columns={'Borough\n':'Borough'}, inplace=True)
df.rename(columns={'Neighborhood\n':'Neighborhood'}, inplace=True)
df['Borough'] = df['Borough'].str.replace('\n','')
df['Neighborhood'] = df['Neighborhood'].str.replace('\n','')
df['Postal Code'] = df['Postal Code'].str.replace('\n','')

df.head(12)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [185]:
import numpy as np

# replace "Not assigned" to NaN
df.replace("Not assigned", np.nan, inplace = True)
df.head(12)

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,NaN,NaN
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [186]:

missing_data = df.isnull()
missing_data.head(5)

,Postal Code,Borough,Neighborhood
0,False,True,True
1,False,True,True
2,False,False,False
3,False,False,False
4,False,False,False


In [189]:
##ignore Nan Borough
df=df.dropna(subset=['Borough'], axis=0)
df.head(12)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [204]:
##re indexing
df1=df.reset_index()
df2=df1[['Borough', 'Postal Code', 'Neighborhood']]
df2.head(12)

,Borough,Postal Code,Neighborhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,"Regent Park, Harbourfront"
3,North York,M6A,"Lawrence Manor, Lawrence Heights"
4,Downtown Toronto,M7A,"Queen's Park, Ontario Provincial Government"
5,Etobicoke,M9A,"Islington Avenue, Humber Valley Village"
6,Scarborough,M1B,"Malvern, Rouge"
7,North York,M3B,Don Mills
8,East York,M4B,"Parkview Hill, Woodbine Gardens"
9,Downtown Toronto,M5B,"Garden District, Ryerson"


In [222]:
df2['Borough'].astype("str")
df2['Postal Code'].astype("str")
df2['Neighborhood'].astype("str")

0                                              Parkwoods
1                                       Victoria Village
2                              Regent Park, Harbourfront
3                       Lawrence Manor, Lawrence Heights
4            Queen's Park, Ontario Provincial Government
5                Islington Avenue, Humber Valley Village
6                                         Malvern, Rouge
7                                              Don Mills
8                        Parkview Hill, Woodbine Gardens
9                               Garden District, Ryerson
10                                             Glencairn
11     West Deane Park, Princess Gardens, Martin Grov...
12                Rouge Hill, Port Union, Highland Creek
13                                             Don Mills
14                                      Woodbine Heights
15                                        St. James Town
16                                    Humewood-Cedarvale
17     Eringate, Bloordale Gard

In [221]:
df2.dtypes()

TypeError: 'Series' object is not callable